### CNN(Convolutional Neural Network : 합성곱 신경망)

- 입력 데이터인 이미지의 성질을 이용해서 파라미터의 수를 줄인다
- 반복되는 합성곱 층과 풀링 층
    - 합성곱 층 : 이미지에 커널(필터)을 적용해서 이미지의 특징량을 추출하는 역할을 담당하는 층
        - 최적화가 필요한 가중치 파라미터의 수는 이미지의 크기가 아닌 필터의 크기에 의존하므로 MLP와 달리 이미지의 크기가 커져도 파라미터는 증가하지 않는다
    - 풀링 층 : 이미지를 축소하는 것 같은 층, 작은 변화에 민감하게 반응하지 않게 하는(Robust) 역할 담당

In [1]:
# 데이터 임포트

from tensorflow.python.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


170500096/170498071 [==============================] - 144s 1us/step


In [8]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [9]:
# 모델이 다루기 쉬운 크기로 데이터 변형

# 임포트한 데이터 크기 확인
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [10]:
# 데이터의 스케일 변환과 클래스 레이블을 원-핫벡터로 변환
from tensorflow.python.keras.utils import to_categorical

# 특징량 정규화
x_train = x_train/255.
x_test = x_test/255.

# 클래스 레이블을 원-핫벡터화
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [11]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
# 모델 구축 준비
from tensorflow.python.keras.models import Sequential

model = Sequential()

In [14]:
# 합성곱 층 추가

# MLP에서는 네트워크 구축에 Dense 레이어를 사용했지만, 
# CNN의 합성곱 층은 Conv2D 레이어를 사용한다

from tensorflow.python.keras.layers import Conv2D

model.add(
    Conv2D(
        filters = 32,           # filters : 출력의 채널 수(특징맵의 수)
        input_shape = (32, 32, 3),
        kernel_size = (3, 3),   # kernel_size : 커널의 크기
        strides = (1, 1),       # strides : 커널을 옆으로 이동시키는 폭
        padding = 'same',       # 제로 패딩(입력과 출력의 크기를 같게 하고 싶은 경우)
        activation = 'relu'
    )
)

model.add(
    Conv2D(
        filters = 32,
        kernel_size = (3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu'
    )
)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
# 풀링 층 추가
from tensorflow.python.keras.layers import MaxPooling2D

# 2x2 크기의 최대풀링 층 추가
model.add(MaxPooling2D(pool_size = (2, 2)))

In [16]:
# 드롭아웃층 추가
# 드롭아웃 : 한 층의 뉴런 중 몇 개를 임의로 무효화해서 학습을 진행하는 것
#            파라미터가 많고 표현력이 높은 네트워크의 자유도를 억제
#            → 모델의 강건성을 높인다

from tensorflow.python.keras.layers import Dropout

# 학습할 때 뉴런의 0.25를 임의로 무효화
model.add(Dropout(0.25))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
# 합성곱 층과 풀링 층 추가

model.add(
    Conv2D(
        filters = 64,
        kernel_size = (3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu'
    )
)

model.add(
    Conv2D(
        filters = 64,
        kernel_size = (3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu'
    )
)

model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [ ]:
# 완전연결 계층 추가

# 그러나 합성곱 층과 풀링 층의 출력은 형식이 다르기 때문에
# 완전연결 계층에 직접 입력할 수 없다

In [18]:
# 풀링 층을 추가한 후의 모델의 출력 형식 확인

# (None, 8, 8, 64)
# → 4차원의 텐서 (데이터 수, 세로, 가로, 채널 수)
model.output_shape

(None, 8, 8, 64)

In [ ]:
# 완전연결 계층은 2차원의 텐서만 입력으로 받는다
# → 다차원의 텐서를 2차원의 텐서로 전개해주는 플래튼(Flatten) 층 추가 

In [19]:
# 플래튼 층 추가
from tensorflow.python.keras.layers import Flatten

model.add(Flatten())
model.output_shape

(None, 4096)

In [20]:
# 완전연결 계층 추가
from tensorflow.python.keras.layers import Dense

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))

In [21]:
# 작성한 모델에 데이터 적용
from tensorflow.python.keras.callbacks import TensorBoard

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
tsb = TensorBoard(log_dir = './logs')
history_model1 = model.fit(
    x_train,
    y_train,
    batch_size = 32,
    epochs = 20,
    validation_split = 0.2,
    callbacks = [tsb]
)

Train on 40000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
40000/40000 [==============================] - 461s 12ms/sample - loss: 1.5709 - acc: 0.4249 - val_loss: 1.1908 - val_acc: 0.5686
Epoch 2/20
40000/40000 [==============================] - 441s 11ms/sample - loss: 1.1495 - acc: 0.5897 - val_loss: 1.0003 - val_acc: 0.6384
Epoch 3/20
40000/40000 [==============================] - 446s 11ms/sample - loss: 0.9931 - acc: 0.6479 - val_loss: 0.8547 - val_acc: 0.6993
Epoch 4/20
40000/40000 [==============================] - 439s 11ms/sample - loss: 0.8893 - acc: 0.6857 - val_loss: 0.7932 - val_acc: 0.7232
Epoch 5/20
40000/40000 [==============================] - 445s 11ms/sample - loss: 0.8210 - acc: 0.7122 - val_loss: 0.7932 - val_acc: 0.7229
Epoch 6/20
40000/40000 [==============================] - 449s 11ms/sample - loss: 0.7637 - acc: 0.7288 - val_loss: 0.7427 - val_acc: 0.7360
Epoch 7/20
40000/40000 [==============================] 